In [32]:
import pickle
from custom_clss_and_fncs import * 
import os



os.chdir('/DeepenData/Repos/geometric_cobra/')

# Open the file in binary mode
with open('results/data/lista_de_grafos.pkl', 'rb') as file:
      
    # Call load method to deserialze
    list_of_graphs = pickle.load(file)
nx_G = nx.read_gpickle("results/data/nx_G.gpickle")


target_node = 'phe_L_c'
producto_idx = list(nx_G.nodes()).index(target_node)



Train_graphs, Test_graphs = train_test_split(
            list_of_graphs,test_size=0.4,shuffle=True)
batch_size = 3
train_loader = make_loader(Train_graphs, batch_size,num_samples=50)
test_loader  = make_loader(Test_graphs, batch_size, num_samples = 50)
out_channels = 170
hidden_dim   = 170
num_layers   = 3
n_nodes      = list_of_graphs[0].num_nodes
loss_fun  = torch.nn.MSELoss()



In [33]:

class regresor_GIN_FC(torch.nn.Module):
    
    def __init__(
        self, 
        target_node_idx: int,
        n_nodes : int, 
        num_features : int, 
        out_channels: int,
        dropout : float = 0.09, 
        hidden_dim : int = 5, 
        #heads : int = 5,
        LeakyReLU_slope : float = 0.01,

        num_layers: int = 3
    ):
        super(regresor_GIN_FC, self).__init__() # TODO: why SUPER gato? 
        self.n_nodes = n_nodes
        self.dropout = dropout
        self.num_features = num_features
        self.target_node_idx = target_node_idx
        self.out_channels = out_channels
        
        self.GIN_layers =  GIN(in_channels= num_features, hidden_channels= hidden_dim, num_layers= num_layers, out_channels= out_channels, dropout=dropout,  jk='lstm', act='LeakyReLU', act_first = True)#.to('cuda')        
        self.FC1        = Linear(in_features=out_channels, out_features=1, bias=True)#.to('cuda')
        self.FC2        = Linear(in_features=self.n_nodes, out_features=1, bias=True)#.to('cuda')        #self.leakyrelu = LeakyReLU(LeakyReLU_slope).to('cuda')
        self.leakyrelu = LeakyReLU(LeakyReLU_slope)#.to('cuda')
    def forward(
        self, x, 
        edge_index, # 
        batch_size
    ):

     x     = self.GIN_layers(x, edge_index)
     x     = x.reshape(batch_size, self.n_nodes, self.out_channels)
     x     = self.FC1(self.leakyrelu(x))
     x     = x.reshape(batch_size, self.n_nodes, 1)
     #x     = self.FC2(x)
     return  self.FC2(self.leakyrelu(x.squeeze()))#x#[:,self.target_node_idx,:].squeeze()



modelo = regresor_GIN_FC(target_node_idx = producto_idx,n_nodes = n_nodes, num_features = 1, 
                      out_channels = out_channels, hidden_dim =hidden_dim, 
                      num_layers=num_layers, dropout=.1).to('cpu') 

optimizer = torch.optim.Adam(modelo.parameters(),)


data = next(iter(train_loader))
out = modelo(data.x, data.edge_index, batch_size = batch_size)
out

tensor([[0.0121],
        [0.0139],
        [0.0130]], grad_fn=<AddmmBackward0>)